In [1]:
from numba import jit
import MDAnalysis as md
import numpy as np
import matplotlib.pyplot as plt
import string
import os

In [8]:
def extract_contacts (u, lipid, stride=2):
    """Timestep = 500 ps, so striding every 2 frame,
    means taking every 1 ns"""
    GCGR = u.select_atoms('resid 26-421')
    GCGR_BB = GCGR.select_atoms('name BB')
    GCGR_resid = np.unique(GCGR.resids)
    tot_frames = len(u.trajectory[::stride])
    num_resids = GCGR_resid.shape[0]    
    GCGR_resid = np.unique(GCGR.resids)

    C = np.zeros([num_resids, tot_frames])
    for idx, ts in enumerate(u.trajectory[::stride]):
        for indx, r in enumerate(GCGR_resid):
            sel = u.select_atoms('resname {0:s} and global around 7 resid {1:d}'.format(lipid, r))
            num_contacts = np.unique(sel.resids).shape[0]
            C[indx, idx] = num_contacts
    sel = u.select_atoms('resname {0:s}'.format(lipid))
    num_lipids_total = np.unique(sel.resids).shape[0]
    ava_con = (np.average(C, axis=1) / num_lipids_total)
    tot_con = (np.sum(C, axis=1) / num_lipids_total)
    return C, ava_con, tot_con

In [9]:
def residence_time (u, lipid, stride=2):
    GCGR = u.select_atoms('resid 26-421')
    GCGR_BB = GCGR.select_atoms('name BB')
    GCGR_resid = np.unique(GCGR.resids)
    num_resids = GCGR_resid.shape[0]    
    GCGR_resid = np.unique(GCGR.resids)

    C = np.zeros([num_resids])
    for rdx, r in enumerate(GCGR_resid):
        C_res = []
        count = 0
        for fdx, ts in enumerate(u.trajectory[::stride]):
            sel = u.select_atoms('resname {0:s} and global around 7 resid {1:d}'.format(lipid, r))
            num_contacts = np.unique(sel.resids).shape[0]
            if num_contacts > 0 :
                count = count +1
            else:
                C_res.append(count)
                count = 0
        C[rdx] = (np.average(C_res))        
    return C


In [4]:
u = md.Universe('GRO/FA_G-prot_0_nowat_renumbered.gro', 'XTC/FA_G-prot_0_nowat_res_center_fit.xtc')

/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:72: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:72: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:72: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:72: UserWarning: Failed to guess the mass for the following atom types: R
  warnings.warn("Failed to guess the mass for the following atom types: {}".for

In [5]:
res_time = residence_time(u, 'CHOL', stride=1000)

In [7]:
res_time.shape

(396,)

In [ ]:
C, ava_Con, tot_con = extract_contacts(u, 'CHOL', stride=5000)

In [ ]:
plt.plot(np.arange(26,422), ava_Con)

In [ ]:
plt.plot(np.arange(26,422), tot_con)

In [ ]:
C.shape

In [ ]:
plt.plot(np.sum(C, axis=1))